# ツイートデータの取得

In [12]:
# Python 用の OAuth 認証ライブラリ
#pip install requests-oauthlib

# Pythonの形態素解析エンジン
#pip install janome

# Word Cloudのライブラリ
#pip install wordcloud

#  絵文字を扱うためのライブラリ
#pip install emoji --upgrade

import json
import config
from requests_oauthlib import OAuth1Session
from time import sleep
import emoji

# 絵文字を除去する
def remove_emoji(src_str):
    return ''.join(c for c in src_str if c not in emoji.UNICODE_EMOJI)

# APIキー設定(別ファイルのconfig.pyで定義しています)
CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
ATS = config.ACCESS_TOKEN_SECRET

# 認証処理
twitter = OAuth1Session(CK, CS, AT, ATS)  

# タイムライン取得エンドポイント
url = "https://api.twitter.com/1.1/statuses/user_timeline.json"  

# パラメータの定義
params = {'screen_name': 'actor_satojiro',
          'exclude_replies': True,
          'include_rts': False,
          'count': 200}

# 出力先ファイル
f_out = open('./output/tweet_data', 'w')

for j in range(100):
    res = twitter.get(url, params=params)
    if res.status_code == 200:
        # API残り回数
        limit = res.headers['x-rate-limit-remaining']
        print("API remain: " + limit)
        if limit == 1:
            sleep(60*15)

        n = 0
        timeline = json.loads(res.text)
        # 各ツイートの本文を表示
        for i in range(len(timeline)):
            if i != len(timeline)-1:
                # 絵文字があると、wordcloudが使用できないため、削除する
                f_out.write(remove_emoji(timeline[i]['text']) + '\n')
            else:
                # 絵文字があると、wordcloudが使用できないため、削除する
                f_out.write(remove_emoji(timeline[i]['text']) + '\n')
                # 一番最後のツイートIDをパラメータmax_idに追加
                params['max_id'] = timeline[i]['id']-1

f_out.close()

API remain: 799
API remain: 798
API remain: 797
API remain: 796
API remain: 795
API remain: 794
API remain: 793
API remain: 792
API remain: 791
API remain: 790
API remain: 789
API remain: 788
API remain: 787
API remain: 786
API remain: 785
API remain: 784
API remain: 783
API remain: 782
API remain: 781
API remain: 780
API remain: 779
API remain: 778
API remain: 777
API remain: 776
API remain: 775
API remain: 774
API remain: 773
API remain: 772
API remain: 771
API remain: 770
API remain: 769
API remain: 768
API remain: 767
API remain: 766
API remain: 765
API remain: 764
API remain: 763
API remain: 762
API remain: 761
API remain: 760
API remain: 759
API remain: 758
API remain: 757
API remain: 756
API remain: 755
API remain: 754
API remain: 753
API remain: 752
API remain: 751
API remain: 750
API remain: 749
API remain: 748
API remain: 747
API remain: 746
API remain: 745
API remain: 744
API remain: 743
API remain: 742
API remain: 741
API remain: 740
API remain: 739
API remain: 738
API rema

# python wordcloud

In [15]:
#　名詞だけ抽出してワードクラウド表示

# coding:utf-8
import csv
from janome.tokenizer import Tokenizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from bs4 import BeautifulSoup
from collections import Counter, defaultdict

# 名詞だけ抽出、単語をカウント
def counter(texts):
    t = Tokenizer()
    words_count = defaultdict(int)
    words = []
    for text in texts:
        tokens = t.tokenize(text)
        for token in tokens:
            # 品詞から名詞だけ抽出
            pos = token.part_of_speech.split(',')[0]
            if pos in ['名詞']:
                # 必要ない単語を省く(実際の結果を見てから不必要そうな単語を記載しました)
                if token.base_form not in ["こと", "よう", "そう", "これ", "それ"]:
                    words_count[token.base_form] += 1
                    words.append(token.base_form)
    return words_count, words

with open('./output/tweet_data', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    texts = []
    for row in reader:
        if(len(row) > 0):
            text = row[0].split('http')
            texts.append(text[0])

words_count, words = counter(texts)
text = ' '.join(words)

# fontは自分の端末にあるものを使用する
fpath = "./font/NotoSansCJKjp-Regular.otf"
wordcloud = WordCloud(background_color="white",
                      font_path=fpath, width=900, height=500).generate(text)

wordcloud.to_file("./output/wordcloud_sample.png")
